In [18]:
import os 

import fiona #must be import before geopandas
import geopandas as gpd
import rasterio
#import rioxarray as xr
import rioxarray

import re
import rtree
import shapely
import pickle

#from cartopy import crs
import collections
import cv2
import math
from glob import glob
from tqdm.notebook import tqdm_notebook

# Standard packages
import tempfile
import warnings
import urllib
import shutil

# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import data_eng.az_proc as ap
import data_eng.form_calcs as fc

from lxml.etree import Element,SubElement,tostring
import xml.dom.minidom
from xml.dom.minidom import parseString
import xml.etree.ElementTree as et
from xml.dom import minidom

#import requests
from PIL import Image
from io import BytesIO
import tqdm
from skimage.metrics import structural_similarity as compare_ssim
import imutils

import psutil

## File Paths

In [4]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

tile_names_tile_urls_complete_array = np.load("image_download_azure/tile_name_tile_url_complete_array.npy")

tiles_labeled = "tile_name_tile_url_labeled.npy"
tiles_labeled_from_complete_set = np.load(tiles_labeled)

tracker_file_path = 'outputs/tile_img_annotation_annotator.npy'
tile_img_annotation = np.load(tracker_file_path)
tile_img_annotation_annotator = np.load("outputs/tile_img_annotation_annotator.npy")

tiles_errors = 'tile_name_tile_url_error_downloading.npy'
tiles_errors = np.load(tiles_errors)

#create folder to hold tiles in completed dataset

tiles_complete_dataset_path = os.path.join(parent_directory,"complete_dataset","tiles")
tiles_xml_complete_dataset_path = os.path.join(parent_directory,"complete_dataset","tiles_xml")
os.makedirs(tiles_complete_dataset_path, exist_ok=True)

tile_names_tile_urls_complete_array_unique_standard_tile_names = np.load("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy")
image_characteristics = pd.read_csv("image_characteristics.csv")
tile_names_tile_urls_complete_array_unique_standard_tile_names = np.load("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy")

#NAIP quad map
quad_indicies_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/AST Datasets/MapIndices_National_GDB/MapIndices_National_GDB.gdb"
#fiona.listlayers(quad_indicies_path)

## File Paths

In [ ]:
###CONFIRM CHIP X AND Y INDEXES

In [7]:
tile_name = os.listdir(tiles_complete_dataset_path)[0]

In [33]:
dataset = rasterio.open(os.path.join(tiles_complete_dataset_path, os.listdir(tiles_complete_dataset_path)[0]))
dataset.crs

"""
#tile_height,  tile_width,  tile_channels = dataset.shape
left = dataset.bounds[0]
right = dataset.bounds[2]
bottom = dataset.bounds[1]
top = dataset.bounds[3]
x = dataset.shape[0] 
y = dataset.shape[1] 
np.linspace(left, right, num=x-1)# endpoint=True, retstep=False, dtype=None, axis=0)
"""



CRS.from_epsg(26914)

In [51]:
y0 = 0
x0 = 0
item_dim = 512

x[y0*item_dim:y0*item_dim+item_dim, x0*(item_dim):x0*(item_dim)+item_dim]

array([[-99.25351586, -99.25350991, -99.25350396, ..., -99.25048646,
        -99.25048051, -99.25047456],
       [-99.25351586, -99.25350991, -99.25350396, ..., -99.25048646,
        -99.25048051, -99.25047456],
       [-99.25351586, -99.25350991, -99.25350396, ..., -99.25048646,
        -99.25048051, -99.25047456],
       ...,
       [-99.25351586, -99.25350991, -99.25350396, ..., -99.25048646,
        -99.25048051, -99.25047456],
       [-99.25351586, -99.25350991, -99.25350396, ..., -99.25048646,
        -99.25048051, -99.25047456],
       [-99.25351586, -99.25350991, -99.25350396, ..., -99.25048646,
        -99.25048051, -99.25047456]])

In [14]:
tile.coords
SE coordinates # min 
x*item_dim, y*item_dim
NW coordinates #max
clip based on max min
#https://automating-gis-processes.github.io/CSC18/lessons/L6/clipping-raster.html

AttributeError: 'DatasetReader' object has no attribute 'coords'

In [ ]:
#specify folder that holds tiles in completed dataset

#unique positive jpgs (file names with the file extension)
unique_positive_jpgs = fc.unique_positive_jpgs_from_parent_directory(args.parent_directory)

image_characteristics = fc.image_characteristics(complete_dataset_tiles_folder_path, unique_positive_jpgs)

image_characteristics.to_csv('image_characteristics.csv')
counterin = 0
counternot = 0 

#Check to see how many images are not yet in the image characteristics folder (not verified)
for unique_jpg in unique_positive_jpgs[:,0]:
    if image_characteristics['six_digit_chip_name'].isin([unique_jpg]).any():
        counterin += 1
    if not image_characteristics['six_digit_chip_name'].isin([unique_jpg]).any():
        counternot += 1
print("images included in the image characteristics csv ",counterin, \
      "images not included in the image characteristics csv \ left to be verified", counternot)

In [ ]:
#for unique_standard_tile_names in tile_names_tile_urls_complete_array_unique_standard_tile_names[:,2]:
#    if unique
tiles_with_no_images = []
tiles_with_images = []
for standard_tile_name in tqdm_notebook(image_characteristics.standard_tile_name.unique()): #Iterate over all the possible tiles that could be included in the dataset
    print(standard_tile_name)
    #get the image characteristics for the images corresponding to each tile
    images_in_tile = image_characteristics.loc[image_characteristics.standard_tile_name == standard_tile_name] #get the annotated images corresponding to ech time
    indicies = images_in_tile.index
    
    #get the characteristics for each time
    da = xr.open_rasterio(os.path.join(tiles_complete_dataset_path, standard_tile_name +".tif"))
    tile_band, tile_height, tile_width = da.shape[0], da.shape[1], da.shape[2]
    #make the xml for each tile
    generate_xml(standard_tile_name, tiles_complete_dataset_path, tiles_xml_complete_dataset_path, tile_band, tile_height, tile_width)
    #add data from each image xml to the corresponding tile xml
    #for i in indicies:
    #    xml_path, root = get_xml_path_from_root_jpg(images_in_tile, i)
    #if not images_in_tile.empty:

In [ ]:
da.crs